In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window
import pyspark.sql.functions as func

In [2]:
game = spark.read.csv("/FileStore/tables/FileStore/tables/game.csv", header = True)	
team = spark.read.csv("/FileStore/tables/FileStore/tables/team_info.csv", header = True)	
stats = spark.read.csv("/FileStore/tables/FileStore/tables/game_teams_stats.csv", header = True)	


In [3]:
game.printSchema()

root
-- game_id: string (nullable = true)
-- season: string (nullable = true)
-- type: string (nullable = true)
-- date_time: string (nullable = true)
-- away_team_id: string (nullable = true)
-- home_team_id: string (nullable = true)
-- away_goals: string (nullable = true)
-- home_goals: string (nullable = true)
-- outcome: string (nullable = true)
-- home_rink_side_start: string (nullable = true)
-- venue: string (nullable = true)
-- venue_link: string (nullable = true)
-- venue_time_zone_id: string (nullable = true)
-- venue_time_zone_offset: string (nullable = true)
-- venue_time_zone_tz: string (nullable = true)

In [4]:
game.show(5)

+----------+--------+----+----------+------------+------------+----------+----------+------------+--------------------+--------------------+-------------------+------------------+----------------------+------------------+
 game_id| season|type| date_time|away_team_id|home_team_id|away_goals|home_goals| outcome|home_rink_side_start| venue| venue_link|venue_time_zone_id|venue_time_zone_offset|venue_time_zone_tz|
+----------+--------+----+----------+------------+------------+----------+----------+------------+--------------------+--------------------+-------------------+------------------+----------------------+------------------+
2012030221|20122013| P|2013-05-16| 3| 6| 2| 3| home win OT| left| TD Garden|/api/v1/venues/null| America/New_York| -4| EDT|
2012030222|20122013| P|2013-05-19| 3| 6| 2| 5|home win REG| left| TD Garden|/api/v1/venues/null| America/New_York| -4| EDT|
2012030223|20122013| P|2013-05-21| 6| 3| 2| 1|away win REG| right|Madison Square Ga...|/api/v1/venues/null| America/New_York| -4| EDT|
2012030224|20122013| P|2013-05-23| 6| 3| 3| 4| home win OT| right|Madison Square Ga...|/api/v1/venues/null| America/New_York| -4| EDT|
2012030225|20122013| P|2013-05-25| 3| 6| 1| 3|home win REG| left| TD Garden|/api/v1/venues/null| America/New_York| -4| EDT|
+----------+--------+----+----------+------------+------------+----------+----------+------------+--------------------+--------------------+-------------------+------------------+----------------------+------------------+
only showing top 5 rows

In [5]:
game.createOrReplaceTempView("game")

In [6]:
# Question 5.1
game_new = spark.sql("SELECT game_id, CAST(season AS integer), CAST(away_goals AS integer), CAST(home_goals AS integer) FROM game")
# game_new.printSchema()
q5_1DF = game_new.withColumn("total_goals", game_new.away_goals + game_new.home_goals)
q5_1DF.drop('away_goals','home_goals').show(5)

+----------+--------+-----------+
 game_id| season|total_goals|
+----------+--------+-----------+
2012030221|20122013| 5|
2012030222|20122013| 7|
2012030223|20122013| 3|
2012030224|20122013| 7|
2012030225|20122013| 4|
+----------+--------+-----------+
only showing top 5 rows

In [7]:
# # Question 5.2
# q5_2DF = q5_1DF.sort(asc("season", "total_goals"))
# # groupBy("season").agg("total_goals")collect()
# # groupBy("season").agg("total_goals")
# q5_2DF.show(20)

In [8]:
# Question 5.2
q5_1DF.createOrReplaceTempView("q52")
a=spark.sql("select * from q52").toPandas()
a.head(200)

Out[ 7 ]: 
 game_id season away_goals home_goals total_goals
0 2012030221 20122013 2 3 5
1 2012030222 20122013 2 5 7
2 2012030223 20122013 2 1 3
3 2012030224 20122013 3 4 7
4 2012030225 20122013 1 3 4
5 2012030311 20122013 3 0 3
6 2012030312 20122013 6 1 7
7 2012030313 20122013 1 2 3
8 2012030314 20122013 0 1 1
9 2012030231 20122013 1 4 5
10 2012030232 20122013 4 1 5
11 2012030233 20122013 1 3 4
12 2012030234 20122013 0 2 2
13 2012030235 20122013 1 4 5
14 2012030236 20122013 4 3 7
15 2012030237 20122013 1 2 3
16 2012030121 20122013 4 2 6
17 2012030122 20122013 1 3 4
18 2012030123 20122013 1 6 7
19 2012030124 20122013 2 3 5
20 2012030125 20122013 6 1 7
21 2012030151 20122013 1 2 3
22 2012030152 20122013 2 5 7
23 2012030153 20122013 2 3 5
24 2012030154 20122013 3 0 3
25 2012030155 20122013 1 5 6
26 2012030181 20122013 1 2 3
27 2012030182 20122013 1 2 3
28 2012030183 20122013 0 1 1
29 2012030184 20122013 3 4 7
.. ... ... ... ... ...
170 2015030163 20152016 3 2 5
171 2015030164 20152016 4 3 7
172 2015030165 20152016 4 3 7
173 2015030166 20152016 3 6 9
174 2015030167 20152016 2 3 5
175 2014030221 20142015 2 1 3
176 2014030222 20142015 2 3 5
177 2014030223 20142015 0 1 1
178 2014030224 20142015 1 2 3
179 2014030225 20142015 1 2 3
180 2014030226 20142015 4 3 7
181 2014030227 20142015 1 2 3
182 2014030211 20142015 2 1 3
183 2014030212 20142015 6 2 8
184 2014030213 20142015 1 2 3
185 2014030214 20142015 6 2 8
186 2014030215 20142015 1 2 3
187 2014030216 20142015 1 4 5
188 2016030131 20162017 2 3 5
189 2016030132 20162017 4 3 7
190 2016030133 20162017 3 4 7
191 2016030134 20162017 5 4 9
192 2016030135 20162017 1 2 3
193 2016030136 20162017 2 1 3
194 2016030141 20162017 1 3 4
195 2016030142 20162017 1 4 5
196 2016030143 20162017 5 4 9
197 2016030144 20162017 4 5 9
198 2016030145 20162017 2 5 7
199 2014030141 20142015 4 1 5

[200 rows x 5 columns]

In [9]:
q5_2DF = spark.sql("SELECT game_id, total_goals, season FROM q52 order by season asc")
q5_2DF.show(200)

+----------+-----------+--------+
 game_id|total_goals| season|
+----------+-----------+--------+
2012030211| 5|20122013|
2012020573| 6|20122013|
2012030212| 7|20122013|
2012030222| 7|20122013|
2012030213| 3|20122013|
2012030224| 7|20122013|
2012030214| 10|20122013|
2012030311| 3|20122013|
2012030215| 8|20122013|
2012030313| 3|20122013|
2012030141| 5|20122013|
2012030231| 5|20122013|
2012030142| 6|20122013|
2012030233| 4|20122013|
2012030143| 7|20122013|
2012030235| 5|20122013|
2012030144| 7|20122013|
2012030237| 3|20122013|
2012030145| 3|20122013|
2012030122| 4|20122013|
2012030146| 3|20122013|
2012030124| 5|20122013|
2012030147| 9|20122013|
2012030151| 3|20122013|
2012030241| 2|20122013|
2012030153| 5|20122013|
2012030242| 7|20122013|
2012030155| 6|20122013|
2012030243| 3|20122013|
2012030182| 3|20122013|
2012030244| 3|20122013|
2012030184| 7|20122013|
2012030245| 3|20122013|
2012030186| 3|20122013|
2012030246| 3|20122013|
2012030162| 9|20122013|
2012030247| 3|20122013|
2012030164| 5|20122013|
2012030411| 7|20122013|
2012030166| 7|20122013|
2012030412| 3|20122013|
2012030111| 5|20122013|
2012030413| 2|20122013|
2012030113| 9|20122013|
2012030414| 11|20122013|
2012030115| 4|20122013|
2012030415| 4|20122013|
2012030131| 4|20122013|
2012030416| 5|20122013|
2012030133| 7|20122013|
2012030171| 4|20122013|
2012030135| 3|20122013|
2012030172| 5|20122013|
2012030137| 5|20122013|
2012030173| 7|20122013|
2012030322| 6|20122013|
2012030174| 7|20122013|
2012030324| 5|20122013|
2012020715| 4|20122013|
2012020225| 5|20122013|
2012020142| 6|20122013|
2012020122| 3|20122013|
2012020280| 10|20122013|
2012020510| 7|20122013|
2012020584| 4|20122013|
2012020116| 5|20122013|
2012020230| 3|20122013|
2012020705| 7|20122013|
2012020211| 10|20122013|
2012020347| 3|20122013|
2012020157| 1|20122013|
2012020632| 7|20122013|
2012020033| 5|20122013|
2012020205| 3|20122013|
2012020178| 6|20122013|
2012020231| 7|20122013|
2012020363| 3|20122013|
2012020161| 5|20122013|
2012020496| 9|20122013|
2012020059| 5|20122013|
2012020714| 4|20122013|
2012020701| 5|20122013|
2012020128| 4|20122013|
2012020239| 5|20122013|
2012020064| 5|20122013|
2012020651| 6|20122013|
2012020697| 5|20122013|
2012020065| 5|20122013|
2012020606| 5|20122013|
2012020364| 8|20122013|
2012020154| 2|20122013|
2012020439| 6|20122013|
2012020247| 4|20122013|
2012020204| 3|20122013|
2012020592| 5|20122013|
2012020141| 9|20122013|
2012020680| 4|20122013|
2012020590| 6|20122013|
2012020210| 8|20122013|
2012020050| 6|20122013|
2012020428| 4|20122013|
2012020552| 3|20122013|
2012020707| 9|20122013|
2012020521| 9|20122013|
2012020293| 4|20122013|
2012020708| 5|20122013|
2012020010| 5|20122013|
2012020717| 3|20122013|
2012020131| 6|20122013|
2012020108| 6|20122013|
2012020337| 5|20122013|
2012020298| 3|20122013|
2012020655| 12|20122013|
2012020026| 9|20122013|
2012020268| 9|20122013|
2012020504| 8|20122013|
2012030223| 3|20122013|
2012020604| 5|20122013|
2012030312| 7|20122013|
2012020047| 4|20122013|
2012030232| 5|20122013|
2012020391| 5|20122013|
2012030236| 7|20122013|
2012020698| 4|20122013|
2012030123| 7|20122013|
2012020041| 3|20122013|
2012030152| 7|20122013|
2012020022| 7|20122013|
2012030181| 3|20122013|
2012020557| 4|20122013|
2012030185| 5|20122013|
2012020596| 3|20122013|
2012030163| 4|20122013|
2012020005| 3|20122013|
2012030167| 5|20122013|
2012020130| 5|20122013|
2012030114| 10|20122013|
2012020621| 2|20122013|
2012030132| 1|20122013|
2012020076| 5|20122013|
2012030136| 1|20122013|
2012020636| 7|20122013|
2012030323| 4|20122013|
2012020382| 7|20122013|
2012020577| 8|20122013|
2012020145| 7|20122013|
2012020511| 7|20122013|
2012020462| 7|20122013|
2012020495| 3|20122013|
2012020668| 3|20122013|
2012020358| 7|20122013|
2012020216| 5|20122013|
2012020663| 5|20122013|
2012020499| 10|20122013|
2012020476| 11|20122013|
2012020605| 2|20122013|
2012020587| 5|20122013|
2012020338| 2|20122013|
2012020003| 7|20122013|
2012020359| 9|20122013|
2012020681| 5|20122013|
2012020322| 9|20122013|
201202

In [10]:
q5_2DF = spark.sql("SELECT game_id, total_goals, season, DENSE_RANK(total_goals) over (partition by season order by total_goals ASC, season ASC) rank FROM q52")
q5_2DF.show(200)
# a=q5_2DF.where(q5_2DF.season != "20152016").toPandas()
# a.head()

+----------+-----------+--------+----+
 game_id|total_goals| season|rank|
+----------+-----------+--------+----+
2017030171| 1|20172018| 1|
2017030174| 1|20172018| 1|
2017021076| 1|20172018| 1|
2017020918| 1|20172018| 1|
2017020795| 1|20172018| 1|
2017020456| 1|20172018| 1|
2017020723| 1|20172018| 1|
2017020451| 1|20172018| 1|
2017020561| 1|20172018| 1|
2017020133| 1|20172018| 1|
2017020278| 1|20172018| 1|
2017020097| 1|20172018| 1|
2017020691| 1|20172018| 1|
2017020255| 1|20172018| 1|
2017021062| 1|20172018| 1|
2017020125| 1|20172018| 1|
2017020546| 1|20172018| 1|
2017020327| 1|20172018| 1|
2017020897| 1|20172018| 1|
2017021141| 1|20172018| 1|
2017020965| 1|20172018| 1|
2017030164| 2|20172018| 2|
2017020624| 2|20172018| 2|
2017021025| 2|20172018| 2|
2017020506| 2|20172018| 2|
2017020012| 2|20172018| 2|
2017020940| 2|20172018| 2|
2017020588| 2|20172018| 2|
2017020208| 2|20172018| 2|
2017020084| 2|20172018| 2|
2017021118| 2|20172018| 2|
2017020917| 2|20172018| 2|
2017020098| 2|20172018| 2|
2017020729| 2|20172018| 2|
2017020956| 2|20172018| 2|
2017021207| 2|20172018| 2|
2017020877| 2|20172018| 2|
2017021054| 2|20172018| 2|
2017020289| 2|20172018| 2|
2017020367| 2|20172018| 2|
2017020780| 2|20172018| 2|
2017020038| 2|20172018| 2|
2017020191| 2|20172018| 2|
2017021043| 2|20172018| 2|
2017020031| 2|20172018| 2|
2017020602| 2|20172018| 2|
2017020290| 2|20172018| 2|
2017020344| 2|20172018| 2|
2017020183| 2|20172018| 2|
2017020975| 2|20172018| 2|
2017020928| 2|20172018| 2|
2017020215| 2|20172018| 2|
2017020501| 2|20172018| 2|
2017020484| 2|20172018| 2|
2017020557| 2|20172018| 2|
2017030246| 3|20172018| 3|
2017030181| 3|20172018| 3|
2017030184| 3|20172018| 3|
2017030316| 3|20172018| 3|
2017030234| 3|20172018| 3|
2017030172| 3|20172018| 3|
2017030325| 3|20172018| 3|
2017030155| 3|20172018| 3|
2017030226| 3|20172018| 3|
2017020586| 3|20172018| 3|
2017020482| 3|20172018| 3|
2017021046| 3|20172018| 3|
2017020423| 3|20172018| 3|
2017021087| 3|20172018| 3|
2017020138| 3|20172018| 3|
2017020699| 3|20172018| 3|
2017020164| 3|20172018| 3|
2017020952| 3|20172018| 3|
2017020614| 3|20172018| 3|
2017020154| 3|20172018| 3|
2017020810| 3|20172018| 3|
2017020555| 3|20172018| 3|
2017020890| 3|20172018| 3|
2017021267| 3|20172018| 3|
2017021157| 3|20172018| 3|
2017020736| 3|20172018| 3|
2017020156| 3|20172018| 3|
2017020462| 3|20172018| 3|
2017020254| 3|20172018| 3|
2017021204| 3|20172018| 3|
2017020486| 3|20172018| 3|
2017020064| 3|20172018| 3|
2017020261| 3|20172018| 3|
2017020330| 3|20172018| 3|
2017020480| 3|20172018| 3|
2017020626| 3|20172018| 3|
2017020529| 3|20172018| 3|
2017020396| 3|20172018| 3|
2017020621| 3|20172018| 3|
2017020181| 3|20172018| 3|
2017021041| 3|20172018| 3|
2017021104| 3|20172018| 3|
2017020765| 3|20172018| 3|
2017020711| 3|20172018| 3|
2017020545| 3|20172018| 3|
2017020288| 3|20172018| 3|
2017021100| 3|20172018| 3|
2017020580| 3|20172018| 3|
2017021003| 3|20172018| 3|
2017020003| 3|20172018| 3|
2017020212| 3|20172018| 3|
2017020870| 3|20172018| 3|
2017020564| 3|20172018| 3|
2017020528| 3|20172018| 3|
2017020727| 3|20172018| 3|
2017021006| 3|20172018| 3|
2017020537| 3|20172018| 3|
2017020071| 3|20172018| 3|
2017021102| 3|20172018| 3|
2017020932| 3|20172018| 3|
2017020349| 3|20172018| 3|
2017020251| 3|20172018| 3|
2017021160| 3|20172018| 3|
2017020658| 3|20172018| 3|
2017020768| 3|20172018| 3|
2017020386| 3|20172018| 3|
2017020397| 3|20172018| 3|
2017020919| 3|20172018| 3|
2017020015| 3|20172018| 3|
2017021146| 3|20172018| 3|
2017021227| 3|20172018| 3|
2017020195| 3|20172018| 3|
2017020209| 3|20172018| 3|
2017021220| 3|20172018| 3|
2017020893| 3|20172018| 3|
2017020102| 3|20172018| 3|
2017020180| 3|20172018| 3|
2017020141| 3|20172018| 3|
2017020503| 3|20172018| 3|
2017020707| 3|20172018| 3|
2017020307| 3|20172018| 3|
2017020189| 3|20172018| 3|
2017020429| 3|20172018| 3|
2017021145| 3|20172018| 3|
2017020799| 3|20172018| 3|
2017021128| 3|20172018| 3|
2017020498| 3|20172018| 3|
2017020040| 3|20172018| 3|
2017021198| 3|20172018| 3|

In [11]:
#Question 5.3
w = Window.partitionBy("season")

q5_3DF= q5_1DF.select("*",avg("total_goals").over(w).alias("avg"), min("total_goals").over(w).alias("min"),max("total_goals").over(w).alias("max"))
q5_3DF.show()

+----------+--------+----------+----------+-----------+-----------------+---+---+
 game_id| season|away_goals|home_goals|total_goals| avg|min|max|
+----------+--------+----------+----------+-----------+-----------------+---+---+
2017030241|20172018| 0| 7| 7|5.939483394833949| 1| 13|
2017030242|20172018| 4| 3| 7|5.939483394833949| 1| 13|
2017030243|20172018| 4| 3| 7|5.939483394833949| 1| 13|
2017030244|20172018| 0| 4| 4|5.939483394833949| 1| 13|
2017030245|20172018| 3| 5| 8|5.939483394833949| 1| 13|
2017030246|20172018| 3| 0| 3|5.939483394833949| 1| 13|
2017030181|20172018| 3| 0| 3|5.939483394833949| 1| 13|
2017030182|20172018| 3| 2| 5|5.939483394833949| 1| 13|
2017030183|20172018| 1| 8| 9|5.939483394833949| 1| 13|
2017030184|20172018| 1| 2| 3|5.939483394833949| 1| 13|
2017030311|20172018| 4| 2| 6|5.939483394833949| 1| 13|
2017030312|20172018| 6| 2| 8|5.939483394833949| 1| 13|
2017030313|20172018| 4| 2| 6|5.939483394833949| 1| 13|
2017030314|20172018| 4| 2| 6|5.939483394833949| 1| 13|
2017030315|20172018| 2| 3| 5|5.939483394833949| 1| 13|
2017030316|20172018| 0| 3| 3|5.939483394833949| 1| 13|
2017030317|20172018| 4| 0| 4|5.939483394833949| 1| 13|
2017030131|20172018| 4| 3| 7|5.939483394833949| 1| 13|
2017030132|20172018| 5| 4| 9|5.939483394833949| 1| 13|
2017030133|20172018| 3| 2| 5|5.939483394833949| 1| 13|
+----------+--------+----------+----------+-----------+-----------------+---+---+
only showing top 20 rows

In [12]:
# #Question 5.4
# w = Window.partitionBy("season")
# q5_4DF= q5_1DF.select("*", (q5_1DF.home_goals - q5_1DF.away_goals).alias("diff_goals"),avg("total_goals").over(w).alias("avg"))
# q5_4DF.show()

+----------+--------+----------+----------+-----------+----------+-----------------+
 game_id| season|away_goals|home_goals|total_goals|diff_goals| avg|
+----------+--------+----------+----------+-----------+----------+-----------------+
2017030241|20172018| 0| 7| 7| 7|5.939483394833949|
2017030242|20172018| 4| 3| 7| -1|5.939483394833949|
2017030243|20172018| 4| 3| 7| -1|5.939483394833949|
2017030244|20172018| 0| 4| 4| 4|5.939483394833949|
2017030245|20172018| 3| 5| 8| 2|5.939483394833949|
2017030246|20172018| 3| 0| 3| -3|5.939483394833949|
2017030181|20172018| 3| 0| 3| -3|5.939483394833949|
2017030182|20172018| 3| 2| 5| -1|5.939483394833949|
2017030183|20172018| 1| 8| 9| 7|5.939483394833949|
2017030184|20172018| 1| 2| 3| 1|5.939483394833949|
2017030311|20172018| 4| 2| 6| -2|5.939483394833949|
2017030312|20172018| 6| 2| 8| -4|5.939483394833949|
2017030313|20172018| 4| 2| 6| -2|5.939483394833949|
2017030314|20172018| 4| 2| 6| -2|5.939483394833949|
2017030315|20172018| 2| 3| 5| 1|5.939483394833949|
2017030316|20172018| 0| 3| 3| 3|5.939483394833949|
2017030317|20172018| 4| 0| 4| -4|5.939483394833949|
2017030131|20172018| 4| 3| 7| -1|5.939483394833949|
2017030132|20172018| 5| 4| 9| -1|5.939483394833949|
2017030133|20172018| 3| 2| 5| -1|5.939483394833949|
+----------+--------+----------+----------+-----------+----------+-----------------+
only showing top 20 rows

In [13]:
# #Question 5.4
# from pyspark.sql import functions as F
# w = Window.partitionBy("season")
# my_window = Window.partitionBy().orderBy("game_id")
# q5_4DFt =q5_1DF.withColumn("prev_value", F.lag(q5_1DF.total_goals).over(my_window))
# diff = q5_4DFt.total_goals - q5_4DFt.prev_value
# q5_4DF= q5_4DFt.select("*", diff.alias("diff_goals"),avg("total_goals").over(w).alias("avg"))
# q5_4DF.show()

+----------+--------+----------+----------+-----------+----------+----------+-----------------+
 game_id| season|away_goals|home_goals|total_goals|prev_value|diff_goals| avg|
+----------+--------+----------+----------+-----------+----------+----------+-----------------+
2012020001|20122013| 3| 1| 4| null| null|5.398263027295285|
2012020002|20122013| 4| 1| 5| 4| 1|5.398263027295285|
2012020003|20122013| 5| 2| 7| 5| 2|5.398263027295285|
2012020004|20122013| 1| 3| 4| 7| -3|5.398263027295285|
2012020005|20122013| 2| 1| 3| 4| -1|5.398263027295285|
2012020006|20122013| 2| 1| 3| 3| 0|5.398263027295285|
2012020007|20122013| 3| 6| 9| 3| 6|5.398263027295285|
2012020008|20122013| 1| 5| 6| 9| -3|5.398263027295285|
2012020009|20122013| 0| 6| 6| 6| 0|5.398263027295285|
2012020010|20122013| 3| 2| 5| 6| -1|5.398263027295285|
2012020011|20122013| 3| 4| 7| 5| 2|5.398263027295285|
2012020012|20122013| 2| 4| 6| 7| -1|5.398263027295285|
2012020013|20122013| 7| 3| 10| 6| 4|5.398263027295285|
2012020014|20122013| 2| 5| 7| 10| -3|5.398263027295285|
2012020015|20122013| 6| 3| 9| 7| 2|5.398263027295285|
2012020016|20122013| 4| 1| 5| 9| -4|5.398263027295285|
2012020017|20122013| 0| 1| 1| 5| -4|5.398263027295285|
2012020018|20122013| 3| 2| 5| 1| 4|5.398263027295285|
2012020019|20122013| 6| 4| 10| 5| 5|5.398263027295285|
2012020020|20122013| 1| 2| 3| 10| -7|5.398263027295285|
+----------+--------+----------+----------+-----------+----------+----------+-----------------+
only showing top 20 rows

In [14]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window
import pyspark.sql.functions as func
w = Window.partitionBy("season")
q5_4DF= q5_3DF.select( q5_3DF.game_id, q5_3DF.season,(q5_3DF.total_goals - q5_3DF.avg).alias("diff_goals"))
q5_4DF.show()

+----------+--------+--------------------+
 game_id| season| diff_goals|
+----------+--------+--------------------+
2017030241|20172018| 1.0605166051660513|
2017030242|20172018| 1.0605166051660513|
2017030243|20172018| 1.0605166051660513|
2017030244|20172018| -1.9394833948339487|
2017030245|20172018| 2.0605166051660513|
2017030246|20172018| -2.9394833948339487|
2017030181|20172018| -2.9394833948339487|
2017030182|20172018| -0.9394833948339487|
2017030183|20172018| 3.0605166051660513|
2017030184|20172018| -2.9394833948339487|
2017030311|20172018|0.060516605166051285|
2017030312|20172018| 2.0605166051660513|
2017030313|20172018|0.060516605166051285|
2017030314|20172018|0.060516605166051285|
2017030315|20172018| -0.9394833948339487|
2017030316|20172018| -2.9394833948339487|
2017030317|20172018| -1.9394833948339487|
2017030131|20172018| 1.0605166051660513|
2017030132|20172018| 3.0605166051660513|
2017030133|20172018| -0.9394833948339487|
+----------+--------+--------------------+
only showing top 20 rows

In [15]:
# Question 5.5
q5_5DF = q5_4DF.orderBy(q5_4DF.diff_goals.desc()).limit(10)
q5_5DF.show()

+----------+--------+-----------------+
 game_id| season| diff_goals|
+----------+--------+-----------------+
2016020661|20162017| 9.49354593773728|
2016020434|20162017| 8.49354593773728|
2012020306|20122013|7.601736972704715|
2014020881|20142015| 7.56633813495072|
2014020567|20142015| 7.56633813495072|
2016020379|20162017|7.493545937737282|
2016020704|20162017|7.493545937737282|
2016020663|20162017|7.493545937737282|
2017020016|20172018|7.060516605166051|
2017021136|20172018|7.060516605166051|
+----------+--------+-----------------+

In [16]:
team.show(5)

+-------+-----------+------------+---------+------------+----------------+
team_id|franchiseId| shortName| teamName|abbreviation| link|
+-------+-----------+------------+---------+------------+----------------+
 1| 23| New Jersey| Devils| NJD| /api/v1/teams/1|
 4| 16|Philadelphia| Flyers| PHI| /api/v1/teams/4|
 26| 14| Los Angeles| Kings| LAK|/api/v1/teams/26|
 14| 31| Tampa Bay|Lightning| TBL|/api/v1/teams/14|
 6| 6| Boston| Bruins| BOS| /api/v1/teams/6|
+-------+-----------+------------+---------+------------+----------------+
only showing top 5 rows

In [17]:
team.printSchema()

root
-- team_id: string (nullable = true)
-- franchiseId: string (nullable = true)
-- shortName: string (nullable = true)
-- teamName: string (nullable = true)
-- abbreviation: string (nullable = true)
-- link: string (nullable = true)

In [18]:
game.printSchema()

root
-- game_id: string (nullable = true)
-- season: string (nullable = true)
-- type: string (nullable = true)
-- date_time: string (nullable = true)
-- away_team_id: string (nullable = true)
-- home_team_id: string (nullable = true)
-- away_goals: string (nullable = true)
-- home_goals: string (nullable = true)
-- outcome: string (nullable = true)
-- home_rink_side_start: string (nullable = true)
-- venue: string (nullable = true)
-- venue_link: string (nullable = true)
-- venue_time_zone_id: string (nullable = true)
-- venue_time_zone_offset: string (nullable = true)
-- venue_time_zone_tz: string (nullable = true)

In [19]:
# Question 6.1
q6_2DF = team.join(game, team.team_id == game.away_team_id).select(team.teamName).where(((game.venue == "TD Garden") & (game.season == "20122013"))| (game.venue == "TD Garden") & (game.season == "20132014")).distinct()
q6_2DF.show()

+------------+
 teamName|
+------------+
 Stars|
 Kings|
Blue Jackets|
 Sharks|
 Sabres|
 Hurricanes|
 Flames|
 Maple Leafs|
 Avalanche|
 Islanders|
 Devils|
 Oilers|
 Lightning|
 Wild|
 Blackhawks|
 Capitals|
 Blues|
 Canucks|
 Predators|
 Canadiens|
+------------+
only showing top 20 rows

In [20]:
# Question 6.2
q6_2DF = team.join(game, team.team_id == game.away_team_id).select(team.teamName).where(((game.venue == "TD Garden") & (game.season == "20122013"))| (game.venue == "TD Garden") & (game.season == "20132014")).distinct().count()
q6_2DF

Out[ 46 ]: 29

In [21]:
stats.show()

+----------+-------+----+-----+----------+----------------+-----+-----+----+---+----------------------+--------------+--------------------+---------+---------+
 game_id|team_id| HoA| won|settled_in| head_coach|goals|shots|hits|pim|powerPlayOpportunities|powerPlayGoals|faceOffWinPercentage|giveaways|takeaways|
+----------+-------+----+-----+----------+----------------+-----+-----+----+---+----------------------+--------------+--------------------+---------+---------+
2012030221| 3|away|FALSE| OT| John Tortorella| 2| 35| 44| 8| 3| 0| 44.8| 17| 7|
2012030221| 6|home| TRUE| OT| Claude Julien| 3| 48| 51| 6| 4| 1| 55.2| 4| 5|
2012030222| 3|away|FALSE| REG| John Tortorella| 2| 37| 33| 11| 5| 0| 51.7| 1| 4|
2012030222| 6|home| TRUE| REG| Claude Julien| 5| 32| 36| 19| 1| 0| 48.3| 16| 6|
2012030223| 6|away| TRUE| REG| Claude Julien| 2| 34| 28| 6| 0| 0| 61.8| 10| 7|
2012030223| 3|home|FALSE| REG| John Tortorella| 1| 24| 37| 2| 2| 0| 38.2| 7| 9|
2012030224| 6|away|FALSE| OT| Claude Julien| 3| 40| 24| 8| 4| 2| 53.7| 8| 6|
2012030224| 3|home| TRUE| OT| John Tortorella| 4| 32| 40| 8| 4| 1| 46.3| 9| 7|
2012030225| 3|away|FALSE| REG| John Tortorella| 1| 29| 25| 13| 2| 1| 50.9| 5| 3|
2012030225| 6|home| TRUE| REG| Claude Julien| 3| 32| 35| 11| 3| 1| 49.1| 12| 9|
2012030311| 6|away| TRUE| REG| Claude Julien| 3| 30| 19| 17| 4| 0| 66.7| 1| 5|
2012030311| 5|home|FALSE| REG| Dan Bylsma| 0| 29| 34| 28| 4| 0| 33.3| 8| 9|
2012030312| 6|away| TRUE| REG| Claude Julien| 6| 29| 19| 6| 1| 0| 50.9| 2| 5|
2012030312| 5|home|FALSE| REG| Dan Bylsma| 1| 27| 37| 4| 2| 0| 49.1| 12| 5|
2012030313| 5|away|FALSE| OT| Dan Bylsma| 1| 54| 46| 16| 6| 0| 57.3| 8| 10|
2012030313| 6|home| TRUE| OT| Claude Julien| 2| 40| 34| 18| 5| 0| 42.7| 10| 13|
2012030314| 5|away|FALSE| REG| Dan Bylsma| 0| 26| 33| 8| 3| 0| 47.5| 8| 1|
2012030314| 6|home| TRUE| REG| Claude Julien| 1| 24| 25| 8| 3| 0| 52.5| 11| 5|
2012030231| 17|away|FALSE| REG| Mike Babcock| 1| 21| 43| 10| 3| 0| 52.5| 6| 4|
2012030231| 16|home| TRUE| REG|Joel Quenneville| 4| 42| 24| 10| 3| 1| 47.5| 2| 11|
+----------+-------+----+-----+----------+----------------+-----+-----+----+---+----------------------+--------------+--------------------+---------+---------+
only showing top 20 rows

In [22]:
# Window Function Example
# from pyspark.sql.window import Window
# import pyspark.sql.functions as func
# windowSpec = \
#   Window 
#     .partitionBy(q5_1DF.season) \
#     .orderBy(df['revenue'].desc())
# dataFrame = sqlContext.table("productRevenue")
# revenue_difference = \
#   (func.max(dataFrame['revenue']).over(windowSpec) - dataFrame['revenue'])
# dataFrame.select(
#   dataFrame['product'],
#   dataFrame['category'],
#   dataFrame['revenue'],
#   revenue_difference.alias("revenue_difference"))